## Predict 2 meter temperature with sea surface temperature using linear regression
This notebook serves as an example of a basic workflow of data driven forecasting using machine learning with `s2spy` & `lilio` packages. <br>
We will predict temperature in US at seasonal time scales using ERA5 dataset with linear regression (Ridge). <br>

<img src="../assets/concept_test_case.png" alt="usecase" width="500"/>

This recipe includes the following steps:
- Define a calendar (`lilio`)
- Download/load input data (test data, accessible via `era5cli`)
- Map the calendar to the data (`lilio`)
- Train-test split (70%/30%)
- Preprocessing based on the training set (`s2spy`)
- Resample data to the calendar (`lilio`)
- Dimensionality reduction and model training, with cross-validation (`lilio` & `scikit-learn`)

The workflow is illustrated below:

<img src="../assets/regression.PNG" alt="Ridge" width="900"/>

In [ ]:
import lilio
import numpy as np
import xarray as xr
from pathlib import Path
from s2spy import preprocess
from s2spy import RGDR
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

#### Define a calendar with `lilio` to specify time range for targets and precursors.

In [ ]:
# create custom calendar based on the time of interest
calendar = lilio.Calendar(anchor="07-01", allow_overlap=True)
# add target periods
calendar.add_intervals("target", length="30d", gap="1M")
# add precursor periods
periods_of_interest = 4
calendar.add_intervals("precursor", "1M", n=periods_of_interest)

In [ ]:
# check calendar
calendar

#### Load test data SST and (clustered) T2M
We use 63 years (1959-2021) of data.

In [ ]:
# load data
precursor_field = xr.open_dataset('../data/sst_daily_1959-2021_5deg_Pacific_175_240E_25_50N.nc')
target_field = xr.open_dataset('../data/t2m_daily_1959-2021_2deg_clustered_226_300E_30_70N.nc')

In [ ]:
# Convert Klevin to Celcius
precursor_field["sst"] = precursor_field["sst"] - 273.15
target_field["t2m"] = target_field["t2m"] - 273.15

#### Map the calendar to the data
After mapping the calendar to the field, we can visualize our calendar by calling the `visualize` method.

In [ ]:
# map calendar to data
calendar.map_to_data(precursor_field)
calendar.visualize(show_length=True)

Also, we can get a list of all intervals by running the following line. There, you will find the intervals `-1` and `1`, which corresponds to the creation of a precursor interval (negative integer(s)) and a target interval (positive integer(s)), respectively. <br>

For more information about the definition of intervals, and how `lilio` works, please check the [README](https://github.com/AI4S2S/lilio) of `lilio`.

In [ ]:
# show first 5 anchor years in the calendar
calendar.show()[:5]

#### Train-test split based on the anchor years (70%/30% split)
This can be refered to as the "outer cross-validation layer".

In [ ]:
# get 70% of instance as training
years = sorted(calendar.get_intervals().index)
train_samples = round(len(years) * 0.7)
start_year = years[0]

#### Fit preprocessor with training samples and preprocess data
In this step, we remove trend and take anomalies for the precursor field. Note that here we use raw daily data for detrending and taking anomalies. <br>

In general, there are many "flavors" of preprocessing, like when to perform this operation, and in which order do we want to preprocess the data. To improve the transparency and reproducibility of our work, we think it is necessary to standardize these steps. To stick to the best practices, we suggest to preprocess your data in the following way.

In [ ]:
# create preprocessor
preprocessor = preprocess.Preprocessor(
    rolling_window_size=25,
    detrend="linear",
    subtract_climatology=True,
)

# fit preprocessor with training data
preprocessor.fit(
    precursor_field.sel(
        time=slice(str(start_year), str(start_year + train_samples - 1))
    )
)

In [ ]:
# preprocess the whole precursor field
precursor_field_prep = preprocessor.transform(precursor_field)

#### Resample data to the calendar

In [ ]:
precursor_field_resample = lilio.resample(calendar, precursor_field_prep)
target_field_resample = lilio.resample(calendar, target_field)

In [ ]:
# select variables and intervals
precursor_field_sel = precursor_field_resample['sst']
target_series_sel = target_field_resample['t2m'].sel(cluster=3)

#### Train-test split -> dimensionality reduction & model fitting
Train-test split based on the previous split, which can be refered to as "inner cross-validation layer" (outer cv loop -> inner cv loop). <br>
For each split, we will perform dimensionality reduction and fit the model.

For simplicity, in this example we only perform dimensionality reduction with RGDR with only one target interval and one lag. But in practice you can have multiple target intervals and multiple lags. <br>
For more information, here is a nice example about multiple target intervals in the [tutorial notebook](https://github.com/AI4S2S/s2spy/blob/main/docs/notebooks/tutorial_RGDR.ipynb) of RGDR.

In [ ]:
# suppress numpy warning
import warnings
warnings.filterwarnings("ignore")

# cross-validation with Kfold
k_fold_splits = 5
kfold = KFold(n_splits=k_fold_splits)
cv = lilio.traintest.TrainTestSplit(kfold)

# create lists for saving models and predictions
models = []
predictions = []
rmse_train = []
rmse_test = []
train_test_splits = []

# prepare operator for dimensionality reduction
target_intervals = 1
lags = list(np.arange(1, periods_of_interest + 1))

# cross validation based dimensionality reduction and model training
for split, (x_train, x_test, y_train, y_test) in enumerate(cv.split(precursor_field_sel, y=target_series_sel)):
    clusters_train_lags = []
    clusters_test_lags = []
    for lag in lags:
        # log train/test splits with anchor years
        train_test_splits.append({
            "train": x_train.anchor_year.values,
            "test": x_test.anchor_year.values,
        })
        # RGDR
        rgdr = RGDR(
            target_intervals=target_intervals,
            lag=lag,
            eps_km=600,
            alpha=0.05,
            min_area_km2=0
            )
        # fit dimensionality reduction operator RGDR and transform
        clusters_train_lag_xr = rgdr.fit_transform(x_train, y_train)
        clusters_test_lag_xr = rgdr.transform(x_test)
        # convert to numpy array, reshape and append
        clusters_train_lag = clusters_train_lag_xr.to_numpy()
        clusters_train_lag = clusters_train_lag.reshape(len(clusters_train_lag_xr.anchor_year),-1)
        clusters_train_lags.append(clusters_train_lag)
        clusters_test_lag = clusters_test_lag_xr.to_numpy()
        clusters_test_lag = clusters_test_lag.reshape(len(clusters_test_lag_xr.anchor_year),-1)
        clusters_test_lags.append(clusters_test_lag)
    # concatenate lags
    clusters_train = np.concatenate(clusters_train_lags, axis=1)
    clusters_test = np.concatenate(clusters_test_lags, axis=1)
    # train model
    ridge = RidgeCV(alphas=[0.1, 10, 25, 50])
    model = ridge.fit(clusters_train, y_train.sel(i_interval=1))
    # save model
    models.append(model)
    # predict and save results
    prediction = model.predict(clusters_test)
    predictions.append(prediction)
    # calculate and save rmse
    rmse_train.append(mean_squared_error(y_train.sel(i_interval=1),
                                            model.predict(clusters_train)))
    rmse_test.append(mean_squared_error(y_test.sel(i_interval=1),
                                        prediction))

#### Plot the predictions

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
# y_train
plt.plot(y_train.anchor_year, y_train.sel(i_interval=1), "b", label = "y_train")
plt.plot(y_train.anchor_year, model.predict(clusters_train), "b--", label = "training")
# y_test
plt.plot(y_test.anchor_year, y_test.sel(i_interval=1), "r", label = "y_test")
plt.plot(y_test.anchor_year, prediction, "r--", label = "prediction")
plt.xlabel("years")
plt.ylabel("deg C")
plt.legend()
plt.show()

#### Plot the RMSE for both training and testing for each fold

In [ ]:
fig = plt.figure()
xrange = np.arange(1, k_fold_splits + 1)
plt.plot(xrange, rmse_train, "b--", label = "train loss")
plt.plot(xrange, rmse_test, "r", label = "test loss")
ax = fig.gca()
ax.set_xticks(xrange)
plt.xlabel("Fold")
plt.ylabel("RMSE")
plt.legend()
plt.show()